In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


In [25]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()


In [26]:
# 프롬프트 템플릿 정의
template = """다음 정보를 사용하여 질문에 답변하세요:

{context}

질문: {question}
이전 대화:
{chat_history}

답변:"""

PROMPT = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=template
)

In [27]:
# 메모리 초기화
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [37]:
# 체인 수동 구현
def retrieve_docs(query):
    return retriever.get_relevant_documents(query)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_answer(input_dict):
    context = input_dict["context"]
    question = input_dict["question"]
    chat_history = input_dict["chat_history"]
    
    prompt = PROMPT.format(context=context, question=question, chat_history=chat_history)
    response = llm.invoke(prompt)
    # 응답에서 문자열 내용만 추출
    return response.content if hasattr(response, 'content') else str(response)

In [38]:
chain = (
    {
        "context": RunnableLambda(retrieve_docs) | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"]
    }
    | RunnableLambda(generate_answer)
)

In [39]:
# 질문 및 답변 함수
def ask_question(question):
    result = chain.invoke(question)
    # 결과가 문자열이 아닌 경우 문자열로 변환
    result_str = result if isinstance(result, str) else str(result)
    memory.save_context({"input": question}, {"output": result_str})
    return result_str

In [41]:
# 질문하기
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

# chain.invoke("Aaronson 은 유죄인가요?")
# ask_question("Aaronson 은 유죄인가요?")

for question in questions:
    answer = ask_question(question)
    print(f"질문: {question}")
    print(f"답변: {answer}\n")


질문: Aaronson 은 유죄인가요?
답변: 제공된 텍스트에는 Aaronson이라는 사람에 대한 언급이 없습니다. 따라서 Aaronson이 유죄인지 아닌지 판단할 수 없습니다.


질문: 그가 테이블에 어떤 메시지를 썼나요?
답변: 제공된 텍스트에는 윈스턴이 테이블에 메시지를 썼다는 내용이 없습니다.  텍스트는 윈스턴이 체스트넛 트리 카페에 앉아 빅 브라더의 포스터를 바라보고 있는 장면을 묘사하고 있습니다.


질문: Julia 는 누구인가요?
답변: 제공된 텍스트에 따르면 줄리아는 윈스턴이 사랑하는 사람입니다.  그는 그녀의 존재에 대한 압도적인 환각을 경험했고, 그녀가 여전히 살아있고 도움이 필요하다고 믿고 있습니다.  그가 그녀의 이름을 부르짖는 순간은 그가 당에 대한 복종을 어기는 행위였습니다.


